# Test experimental model

This notebook tests the model used for experimental cells runs correctly on a simple step stimulus.

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path
import time

import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import numpy as np

import shutil
from pprint import pprint

import multimodalfitting as mf

import neuroplotlib as nplt

%matplotlib notebook 
#notebook

In [ ]:
model_name = "experimental" # "hay", "hay_ais", "hay_ais_hillock"
probe_type = "planar"
base_folder = Path("../..")
cell_models_folder = base_folder / "cell_models"

model_folder = cell_models_folder / f"{model_name}_model"
# data_folder = Path("/gpfs/bbp.cscs.ch/home/mandge/lfpy/multimodalfitting/data/cell1_210301") 
data_folder = base_folder / "data_210916" / "experimental" / "cell1_210301"

In [ ]:
morphology = cell_models_folder / f"{model_name}_model" / "morphology_corrected.swc" #data_folder / "morphology" / "morphology_corrected.swc"
probe_file = data_folder / "extracellular" / "probe.json"
params_release = "experimental_model/parameters_abd_release.json"

In [ ]:
cell = mf.create_experimental_model(morphology_file=morphology,
                                    cell_model_folder=model_folder,
                                    release=True,
                                    v_init=-70, 
                                    abd=False, model_type="LFPy")
cell_abd = mf.create_experimental_model(morphology_file=morphology,
                                    cell_model_folder=model_folder,
                                    release=True,
                                    v_init=-70, 
                                    abd=True, model_type="LFPy")

In [ ]:
# define a probe (this can also be defined externally)
probe = mf.define_electrode(probe_file=probe_file)

# define a test protocol
protocols = mf.define_test_step_protocol(step_amplitude=0.5, tot_duration=500, step_duration=400, probe=probe)

# instantiate simulator
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)
sim_abd = ephys.simulators.LFPySimulator(cell_abd, cvode_active=True, electrode=probe, mechs_folders=model_folder)

In [ ]:
mf.plot_cell(cell, sim, color="b", color_ais="g", detailed=True)
mf.plot_cell(cell_abd, sim, color="b", color_ais="g", color_abd="r", detailed=True)